# Real-time Face Recognition

Let's apply all our previous knowledge into a real-time face recognition. We will try to continuously perform facial recognition on a video stream. In this example I will use an offline video as input, but in reality you can apply it using a webcam, a cctv's live camera feed, etc! 

In [11]:
import cv2
import matplotlib.pyplot as plt
import keras_vggface as kv
import modules.utils as utils
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import nmslib

In [12]:
# Declare a FacePreprocess instance.
from modules.FacePreprocess import FacePreprocess
ssd_model = r'./models/ssd/deploy.prototxt.txt'
ssd_weights = r'./models/ssd/res10_300x300_ssd_iter_140000.caffemodel'
processor = FacePreprocess(ssd_model, ssd_weights)

In [13]:
# Use the facial embedding model you want to use
model = kv.VGGFace(
    model='resnet50', 
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg'
)
input_size = (224, 224)

In [14]:
# importing our nmslib index tree
nmslib_path = './output/large_scale_face_recognition/'

# load id_list
id_list = pd.read_csv(nmslib_path + '/IDlist.csv')

# Euclidean distance
index_l2 = nmslib.init(method='hnsw', space='l2', data_type=nmslib.DataType.DENSE_VECTOR)
index_l2.loadIndex(nmslib_path + 'index_l2.bin')

# Cosine similarity
index_cos = nmslib.init(method='hnsw', space='cosinesimil', data_type=nmslib.DataType.DENSE_VECTOR)
index_cos.loadIndex(nmslib_path + 'index_cos.bin')

For easier evaluation, we will use a video with only one of the subjects (joy).

In [15]:
# original clip from https://youtu.be/Ia3x_X_OX58?si=aA5GdMpRGcCar2xF 
video_path = './dataset/test/test_2/joy.mp4'

## Capture frame-by-frame

We will use openCV's `VideoCapture` which will return the video frame by frame to try and recognize the person in the video. To test the accuracy, we will keep track of the predictions. We will also output a video showing the prediction results. 

In [16]:
results = pd.DataFrame(
    columns = ['count', 'irene', 'seulgi', 'wendy', 'joy', 'yeri'], 
    index = ['l2', 'cosinesimil']
)
results.fillna(0, inplace=True)

`count` will keep track the number of predictions made, and the columns for the other subjects will count how many times where they predicted as the person in the video.

In [17]:
# read: https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html#afec93f94dc6c0b3e28f4dd153bc5a7f0 
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

#### Euclidean dist. `l2`

In [18]:
dist = 'l2'

# load video
cap = cv2.VideoCapture(video_path)

# setup video writer
vid = './output/real_time_face_recognition/resnet50_l2.mp4'
out = cv2.VideoWriter() 
out.open(vid, fourcc, 24.0, (1920, 1080), True) # frame rate, frame size (w, h) must be same as input

# read video frame-by-frame
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        img = frame.copy()
        faces = processor.preproc(img)

        if len(faces)>0:
            for face in faces:
                results['count'][dist] += 1

                # target embeddings
                target = model.predict(utils.resize(face[0], input_size), verbose=False)[0,:]
                target = np.array(target, dtype='f')
                target = np.expand_dims(target, axis=0)

                # predict
                neighbors, distances = index_l2.knnQueryBatch(target, k=1, num_threads=4)[0]
                
                # results
                name = id_list['name'][neighbors[0]]
                results[name][dist] += 1

            # add video frame
            top, bottom, left, right = face[1][0], face[1][1], face[1][2], face[1][3]
            cv2.putText(frame, str(name), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 1) 
        out.write(frame)

        # if you unblock this part a window will pop up and show the frames, but only works if you run with .py 
        # On jupyter notebooks the window will crash after the loop ends and you have to restart the kernel
        # cv2.imshow("frame", frame)
        # cv2.waitKey(1)
        
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

#### Cosine simil. `cosinesimil`

In [19]:
dist = 'cosinesimil'

# load video
cap = cv2.VideoCapture(video_path)

# setup video writer
vid = './output/real_time_face_recognition/resnet50_cosinesimil.mp4'
out = cv2.VideoWriter() 
out.open(vid, fourcc, 24.0, (1920, 1080), True) # frame rate, frame size (w, h) must be same as input

# read video frame-by-frame
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        img = frame.copy()
        faces = processor.preproc(img)

        if len(faces)>0:
            for face in faces:
                results['count'][dist] += 1

                # target embeddings
                target = model.predict(utils.resize(face[0], input_size), verbose=False)[0,:]
                target = np.array(target, dtype='f')
                target = np.expand_dims(target, axis=0)

                # predict
                neighbors, distances = index_cos.knnQueryBatch(target, k=1, num_threads=4)[0]
                
                # results
                name = id_list['name'][neighbors[0]]
                results[name][dist] += 1

            # add video frame
            top, bottom, left, right = face[1][0], face[1][1], face[1][2], face[1][3]
            cv2.putText(frame, str(name), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 1) 
        out.write(frame)

        # if you unblock this part a window will pop up and show the frames, but only works if you run with .py 
        # On jupyter notebooks the window will crash after the loop ends and you have to restart the kernel
        # cv2.imshow("frame", frame)
        # cv2.waitKey(1)
        
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Let's check out the results!

In [20]:
results

,count,irene,seulgi,wendy,joy,yeri
l2,543,0,0,0,543,0
cosinesimil,543,0,0,0,543,0


The test video only contains Joy, so it looks like the prediction results are correct. Using both L2 and Cosine distance achieve similar results. You can also use your webcam camera! Just change the `VideoCapture` input into your webcam, train your own dataset, and you're set.

## Conclusion

In this notebook I showed how to perform facial recognition in real time. This has many real world applications, including surveillance, smart home automatic door, etc. I hope you find this useful for your projects!